In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from numba import jit
class LambdaMART:
    def __init__(self, number_trees=5, maximum_depth=3, lr=1.0):# the program may run slow when the number of tree is high.
        
        """
        number_trees: we set it to default 5. Larger the number of trees slower is the program.
        lr = learning rate by which we train the trees.
        maximum_depth = Number of nodes till where we need to train our model.
        
        """
        self.number_trees = number_trees
        self.maximum_depth = maximum_depth
        self.lr = lr
        self.ndcg_dict = {}
        self.trees = []
        self.gamma = np.zeros((number_trees, 2**(maximum_depth + 1) - 1))
        
        
    def __idcg__(self, rank):
        rank = np.sort(rank)[::-1]
        g = np.arange(1, len(rank)+1)
        a = np.dot((2**rank - 1), 1/np.log2(g + 1))
        return a
        
    @jit# Accelerate the runtime of the function
    def lambda_calc(self, rank, F, query_id):
        log_t = np.argsort(F)[::-1]
        log_f = log_t +1
        # if query id is not in the dictionary we apply the __idcg__ with the rank as the parameter.
        if query_id not in self.ndcg_dict:
            idcg_init = self.__idcg__(rank) 
            
            if idcg_init != 0:
                ndcg_dl = (2**rank - 1)
                ndcg_t = ndcg_dl / idcg_init
                
            else:
                ndcg_t = np.zeros(len(rank))
                
        else:
            ndcg_t = self.ndcg_dict[query_id]
                   
            self.ndcg_dict[query_id] = ndcg_t
        delta_s_min = np.reshape(F, (-1, 1))
        matrix_s = delta_s_min - np.reshape(F, (1, -1))
        ndcg_matrix1 = np.zeros((len(rank), len(rank)))
        
        log_ds = np.log2(1+log_f)
        structure_log = 1 / log_ds
        
        
        for i in range(len(rank)):
            for k in range(i+1, len(rank)):
                
                if rank[i] != rank[k]: # checks whether rank[i] is eaqual to rank[k].
                   
                    log_change = np.copy(structure_log)
                    log_change[i], log_change[k] = log_change[k], log_change[i]
                    
                    if rank[i] < rank[k]:
                        ndcg_matrix1[k, i] = np.dot(ndcg_t, log_change - structure_log)
                    
                    else:
                         ndcg_matrix1[i, k] = np.dot(ndcg_t, log_change - structure_log)
                        
        matrix_r = 1 + np.exp(matrix_s)                
        div_matrix = 1 / matrix_r
        ndcg_matrix_avg = np.abs(ndcg_matrix1)
        matrix_r = ndcg_matrix_avg
        matrix_o = matrix_r* div_matrix
        lambda_M = -matrix_r
        a_init = (1 - div_matrix)
        matrix_r = matrix_r*a_init 
        lambda_t = lambda_M.T
        lambda_M = lambda_M-lambda_t 
        t_lambda =  matrix_r.T
        matrix_r = matrix_r + t_lambda 
        matrix_sum1 = np.sum(lambda_M, axis=0), np.sum(matrix_r, axis=0)
        return matrix_sum1
  
   
    
    def fit(self, X, rank, qid):
        # It gathers the prediction through each and every tree processed by the learning rate
        #Fit the model on the training data. 
        F = np.zeros(np.shape(X)[0])
        for k in range(self.number_trees):
            lambda_array = np.array([])
            omega_array = np.array([])
            for u_query_id in np.unique(qid):
                query_id_lambda, query_id_omega = self.lambda_calc(rank[qid == u_query_id], 
                                                         F[qid == u_query_id], u_query_id)
                lambda_array = np.append(lambda_array, query_id_lambda)
                omega_array = np.append(omega_array, query_id_omega)
            tree_reg = DecisionTreeRegressor(max_depth=self.maximum_depth)
            tree_reg.fit(X, lambda_array)
            self.trees.append(tree_reg)
            leaves = tree_reg.apply(X)
            for branch_id in np.unique(leaves):
                branch_id1 = (leaves == branch_id)
                self.gamma[k, branch_id] = np.sum(lambda_array[branch_id1]) / (np.sum(omega_array[branch_id1]))
                T = self.lr * branch_id1 * self.gamma[k, branch_id]
                F = F+T
                
    def predict(self, X):
        # Predicts score for the test dataset
        F = np.zeros(np.shape(X)[0])
        for k in range(len(self.trees)):
            leaves = self.trees[k].apply(X)
            for branch_id in np.unique(leaves):
                branch_id1 = (leaves == branch_id)
                L_id = self.lr * branch_id1 
                M_id = L_id* self.gamma[k, branch_id]
                F = F + M_id 
        return F

    

In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split

In [5]:
# Evaluating optimization technique - Discounted Cumulative Gain
def dcg(t, k):
    i = np.arange(1, len(t)+1)
    g = (2**t - 1)
    g_calc= g/np.log2(i + 1)
    if k is not None:
        g_calc = g_calc[i <= k]
    return g_calc.sum()

In [6]:
# Evaluating optimization technique - ideal Discounted Cumulative Gain
def idcg(t, k):
    t = np.sort(t)[::-1]
    i = np.arange(1, len(t)+1)
    g = (2**t - 1)
    g_calc = g/np.log2(i + 1)
    if k is not None:
        g_calc = g_calc[i <= k]
    return g_calc.sum()

In [7]:
# Evaluating optimization technique - Normalized Discounted Cumulative Gain
def ndcg(t, k):
    idcg_value = idcg(t, k=k)
    if idcg_value == 0:
        return 0
        
    else:
        return dcg(t, k=k) / idcg_value  

In [8]:
# Evaluating optimization technique - Normalized Discounted Cumulative Gain Mean
def ndcg_mean(t_calc, k):
    ndcg_val = 0
    for qid in t_calc['QId'].unique():
        t = t_calc[t_calc['QId'] == qid]['t']
        ndcg_que = ndcg(t, k=k)
        ndcg_val = ndcg_val + ndcg_que
    return ndcg_val / t_calc['QId'].nunique()

In [9]:
# We load the training dataset by using load_svmlight_file .
# This format has one sample for per line. Features with value zero are not stored here.
data_frame, rank, qid = load_svmlight_file('train.txt', query_id = True)


In [10]:
# We load the test dataset by using load_svmlight_file .
df_test, rank_test, qid_test = load_svmlight_file('test.txt', query_id = True)

In [11]:
#Total number of queries in the dataset
print(f'Total queries in the dataset: {len(np.unique(qid))}')

Total queries in the dataset: 6000


In [12]:
# we select some random queries from the dataset. This is set to 1000.
sample_size = 1000
sample_value = random.sample(list(np.unique(qid)), sample_size)
sample_query = (qid == sample_value[0])

In [63]:
for idx in sample_value[1:]:
    sample_query |= (qid == idx)
df_1 = data_frame[sample_query]
rank_1 = rank[sample_query]
qid_1 = qid[sample_query]

In [13]:
# We split the data into test and train where I specified the size (test size) as 0.2, the rest is training that is the remaining 80 percent.
train_vid, cumulative_idx = train_test_split(np.unique(qid_1), size=0.2)

In [14]:
train_qid1 = (qid_1 == train_vid[0])
for idx in train_vid[1:]:
    train_qid1 |= (qid_1 == idx)

In [15]:
train_cvid1 = (qid_1 == cumulative_idx[0])
for idx in cumulative_idx[1:]:
    train_cvid1 |= (qid_1 == idx)

In [16]:
train_data_frame = df_1[train_qid1]
train_df_rank = rank_1[train_qid1]
query_id_training = qid_1[train_qid1]

data_frame_train1 = df_1[train_cvid1]
cumulative_rank = rank_1[train_cvid1]
queryid_calc = qid_1[train_cvid1]

In [27]:
len(train_df_rank) # length of the rank by the query if of training set.

97032

In [18]:
import warnings
warnings.filterwarnings('ignore')
# We then apply the LambdaMart with the deired input arguments
model = LambdaMART(number_trees=100, maximum_depth=4, lr=0.125)
model.fit(train_data_frame, train_df_rank, query_id_training)# model is then fit into the data frames and the training set

In [19]:
# we predict the model on the traiining set.
predictions = model.predict(data_frame_train1)

In [20]:
result = pd.DataFrame({'negative_value': -predictions, 'QId': queryid_calc, 
                       'Document_Id': np.arange(1, len(queryid_calc)+1), 't': cumulative_rank})
result = result.sort_values(by=['QId', 'negative_value'])


In [21]:
ndcg_mean(result, k=50)

0.3881777197772533

In [22]:
# We predict the model for the test set.
predictions_model = model.predict(df_test)

In [23]:
result1 = pd.DataFrame({'negative_value': -predictions_model, 'QId': qid_test, 'Document_Id': np.arange(1, len(qid_test)+1)})

In [29]:
result1 = result1.sort_values(by=['QId', 'negative_value'])

In [31]:
# The result is printed to the csv file named ranking.csv.
result1[['QId', 'Document_Id']].to_csv('ranking.csv', index=False)